# T81-558: Applications of Deep Neural Networks
**Module 1: Python Preliminaries**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module Video Material

Weekly video update:

* [January 14, 2018 Update for Fall 2018](https://www.youtube.com/watch?v=AuOclsnoe64) 

Main video lecture ([complete YouTube Playlist](https://www.youtube.com/playlist?list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN)):

* [Part 1.1: Course Overview](https://www.youtube.com/watch?v=sRy26qWejOI&index=1&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN)
* [Part 1.2: Machine Learning Background for Deep Learning, Keras and Tensorflow](https://www.youtube.com/watch?v=807L9m5dWng&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN)
* [Part 1.3: Python Anaconda for Deep Learning, Keras and Tensorflow](https://www.youtube.com/watch?v=uOMhboAnVNk&index=3&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN)
* [How to Submit a Module Assignment](https://www.youtube.com/watch?v=hmCGjCVhYNc)

Watch one (or more) of these depending on how you want to setup your Python TensorFlow environment:

* [Installing TensorFlow, Keras, and Python in Windows](https://www.youtube.com/watch?v=z0qhKP2liHs&index=5&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN)
* [Installing TensorFlow, Keras, and Python in Mac](https://www.youtube.com/watch?v=RUs_qR1hKds&index=4&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN)
* [Installing/Using IBM Cognitive Class Labs with TensorFlow/Keras](https://www.youtube.com/watch?v=CyWxFqMVOvg)


# Helpful Functions

You will see these at the top of every module.  These are simply a set of reusable functions that we will make use of.  Each of them will be explained as the semester progresses.  They are explained in greater detail as the course progresses.  Class 4 contains a complete overview of these functions.

In [1]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Course Description

Deep learning is a group of exciting new technologies for neural networks. By using a combination of advanced training techniques neural network architectural components, it is now possible to train neural networks of much greater complexity. This course will introduce the student to deep belief neural networks, regularization units (ReLU), convolution neural networks and recurrent neural networks. High performance computing (HPC) aspects will demonstrate how deep learning can be leveraged both on graphical processing units (GPUs), as well as grids. Deep learning allows a model to learn hierarchies of information in a way that is similar to the function of the human brain. Focus will be primarily upon the application of deep learning, with some introduction to the mathematical foundations of deep learning. Students will use the Python programming language to architect a deep learning model for several of real-world data sets and interpret the results of these networks.

# Assignments

Your grade will be calculated according to the following assignments:

Assignment          |Weight|Title
--------------------|------|-------
Class Participation |   10%|Class attendance and participation
Class Assignments   |   50%|10 small programming assignments (5% each)
Kaggle Project      |   20%|"Kaggle In-Class" project submitted through Kaggle
Final Project       |   20%|A computer security project

The 10 class assignments will be assigned with each of the first 10 modules.  Generally, each module assignment is due just before the following module date.  Refer to syllabus for exact due dates.  The 10 class assignments are submitted using the Python submission script.  Refer to assignment 1 for details.

Module   |Title
---------|--------------
Module 1 |[Simple Test Submission](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class1.ipynb)
Module 2 |[Data Preparation in Pandas](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class2.ipynb)
Module 3 |[Creating Columns in Pandas](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class3.ipynb)
Module 4 |[Regression Neural Network](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class4.ipynb)
Module 5 |[K-Fold Cross-Validation](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class5.ipynb)
Module 6 |[Classification Neural Network](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class6.ipynb)
Module 7 |[Computer Vision Neural Network](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class7.ipynb)
Module 8 |[Building a Kaggle Submission File](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class8.ipynb)
Module 9 |[Exploring Regularization](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class9.ipynb)
Module 10|[Time Series Neural Network](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class10.ipynb)


# Your Instructor: Jeff Heaton

![Jeff Heaton at WUSTL Video Studio](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/jheaton1.png "Jeff Heaton")

I will be your instructor for this course.  A brief summary of my credentials is given here:

* Master of Information Management (MIM), Washington University in St. Louis, MO
* PhD in Computer Science, Nova Southeastern University in Ft. Lauderdale, FL
* [Lead Data Scientist](http://www.rgare.com/knowledge-center/media/articles/rga-where-logic-meets-curiosity), Reinsurance Group of America (RGA)
* Senior Member, IEEE
* jtheaton at domain name of this university
* Other industry certifications: FLMI, ARA, ACS

Social media:

* [Homepage](http://www.heatonresearch.com) - My home page.  Includes my research interests and publications.
* [Linked In](https://www.linkedin.com/in/jeffheaton) - My Linked In profile, feel free to connect.
* [Twitter](https://twitter.com/jeffheaton) - My Twitter feed.
* [Google Scholar](https://scholar.google.com/citations?user=1jPGeg4AAAAJ&hl=en) - My citations on Google Scholar.
* [Research Gate](https://www.researchgate.net/profile/Jeff_Heaton) - My profile/research at Research Gate.
* [Others](http://www.heatonresearch.com/about/) - About me and other social media sites that I am a member of.

# Course Resources

* [IBM Cognitive Class Labs](https://www.datascientistworkbench.com) - Free web based platform that includes Python, Juypter Notebooks, and TensorFlow.  No setup needed.
* [Python Anaconda](https://www.continuum.io/downloads) - Python distribution that includes many data science packages, such as Numpy, Scipy, Scikit-Learn, Pandas, and much more.
* [Juypter Notebooks](http://jupyter.org/) - Easy to use environment that combines Python, Graphics and Text. 
* [TensorFlow](https://www.tensorflow.org/) - Google's mathematics package for deep learning.
* [Kaggle](https://www.kaggle.com/) - Competitive data science.  Good source of sample data.
* [Course GitHub Repository](https://github.com/jeffheaton/t81_558_deep_learning) - All of the course notebooks will be published here.

# What is Deep Learning

The focus of this class is deep learning, which is a very popular type of machine learning that is based upon the original neural networks popularized in the 1980's. There is very little difference between how a deep neural network is calculated compared with the original neural network.  We've always been able to create and calculate deep neural networks.  A deep neural network is nothing more than a neural network with many layers.  While we've always been able to create/calculate deep neural networks, we've lacked an effective means of training them.  Deep learning provides an efficient means to train deep neural networks.

## What is Machine Learning

If deep learning is a type of machine learning, this begs the question, "What is machine learning?"  The following diagram illustrates how machine learning differs from traditional software development.

![ML vs Traditional Software Development](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_1_ml_vs_trad.png "Machine Learning vs Traditional Software Development")

* **Traditional Software Development** - Programmers create programs that specify how to transform input into the desired output.
* **Machine Learning** - Programmers create models that can learn to produce the desired output for given input. This learning fills the traditional role of the computer program. 

Researchers have applied machine learning to many different areas.  This class will explore three specific domains for the application of deep neural networks:

![Application of Machine Learning](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_1_ml_types.png "Application of Machine Learning")

* **Predictive Modeling** - Several named input values are used to predict another named value that becomes the output.  For example, using four measurements of iris flowers to predict the species.  
* **Computer Vision** - The use of machine learning to detect patterns in visual data.  For example, is an image a cat or a dog.
* **Time Series** - The use of machine learning to detect patterns in in time.  Common applications of time series are: financial applications, speech recognition, and even natural language processing (NLP). 

### Regression

Regression is when a model, such as a neural network, accepts input and produces a numeric output.  Consider if you were tasked to write a program that predicted how many miles per gallon (MPG) a car could achieve.  For the inputs you would probably want such features as the weight of the car, the horsepower, how large the engine is, etc.  Your program would be a combination of math and if-statements.  

Machine learning lets the computer learn the "formula" for calculating the MPG of a car, using data.  Consider [this](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/data/auto-mpg.csv) dataset.  We can use regression machine learning models to study this data and learn how to predict the MPG for a car. 

### Classification

The output of a classification model is what class the input belongs to.  For example, consider using four measurements of an iris flower to determine the species that the flower is in.  This dataset could be used to perform [this](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/data/iris.csv).  


## What are Neural Networks

Neural networks are one of the earliest types of machine learning model.  Neural networks were originally introduced in the 1940's and have risen and fallen [several times from popularity](http://hushmagazine.ca/living-2/business/the-believers-the-hidden-story-behind-the-code-that-runs-our-lives). Four researchers have contributed greatly to the development of neural networks.  They have consistently pushed neural network research, both through the ups and downs: 

![Neural Network Luminaries](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_1_luminaries_ann.png "Neural Network Luminaries")

The current luminaries of artificial neural network (ANN) research and ultimately deep learning, in order as appearing in the above picture:

* [Yann LeCun](http://yann.lecun.com/), Facebook and New York University - Optical character recognition and computer vision using convolutional neural networks (CNN).  The founding father of convolutional nets.
* [Geoffrey Hinton](http://www.cs.toronto.edu/~hinton/), Google and University of Toronto. Extensive work on neural networks. Creator of deep learning and early adapter/creator of backpropagation for neural networks.
* [Yoshua Bengio](http://www.iro.umontreal.ca/~bengioy/yoshua_en/index.html), University of Montreal. Extensive research into deep learning, neural networks, and machine learning.  He has so far remained completely in academia.
* [Andrew Ng](http://www.andrewng.org/), Badiu and Stanford University.  Extensive research into deep learning, neural networks, and application to robotics.

## Why Deep Learning?

For predictive modeling neural networks are not that different than other models, such as:

* Support Vector Machines
* Random Forests
* Gradient Boosted Machines

Like these other models, neural networks can perform both **classification** and **regression**.  When applied to relatively low-dimensional predictive modeling tasks, deep neural networks do not necessarily add significant accuracy over other model types.  Andrew Ng describes the advantage of deep neural networks over traditional model types as follows:

![Why Deep Learning?](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_1_why_deep.png "Why Deep Learning")

Neural networks also have two additional significant advantages over other machine learning models:

* **Convolutional Neural Networks** - Can scan an image for patterns within the image.
* **Recurrent Neural Networks** - Can find patterns across several inputs, not just within a single input.

# Python for Deep Learning

Python 3.x is the programming language that will be used for this class.  Python, as a programming language, has the widest support for deep learning.  The three most popular frameworks for deep learning in Python are:

* [TensorFlow](https://www.tensorflow.org/) (Google)
* [MXNet](https://github.com/dmlc/mxnet) (Amazon)
* [Theano](http://deeplearning.net/software/theano/) (University of Montreal) 
* [CNTK](https://cntk.ai/) (Microsoft)

Some references on popular programming languages for AI/Data Science:

* [Popular Programming Languages for AI](https://en.wikipedia.org/wiki/List_of_programming_languages_for_artificial_intelligence)
* [Popular Programming Languages for Data Science](http://www.kdnuggets.com/2014/08/four-main-languages-analytics-data-mining-data-science.html)

# Software Installation
This is a technical class.  You will need to be able to compile and execute Python code that makes use of TensorFlow for deep learning. There are two options to you for accomplish this:

* Install Python, TensorFlow and some IDE (Jupyter, TensorFlow, etc.)
* Use IBM Cognitive Class Labs online

## Installing Python and TensorFlow

It is possible to install and run Python/TensorFlow entirely from your own computer.  Google provides TensorFlow for Windows, Mac and Linux.  Previously, TensorFlow did not support Windows.  However, as of December 2016, TensorFlow supports Windows for both CPU and GPU operation.

The first step is to install Python 3.6.  As of August 2017, this is the latest version of Python 3.  I recommend using the Anaconda release of Python, as it already includes many of the data science related packages that will be needed by this class.  Anaconda directly supports: Windows, Mac and Linux.  Download Anaconda from the following URL:

* [Anaconda](https://www.continuum.io/downloads)

Once Anaconda has been downloaded it is easy to install Jupyter notebooks with the following command:

```
conda install jupyter
```

Once Jupyter is installed, it is started with the following command:

```
jupyter notebook
```

The following packages are needed for this course:

```
conda install scipy
pip install sklearn
pip install pandas
pip install pandas-datareader
pip install matplotlib
pip install pillow
pip install requests
pip install h5py
pip install tensorflow==1.4.0
pip install keras==2.1.2
```

Notice that I am installing as specific version of TensorFlow.  As of the current semester, this is the latest version of TensorFlow.  It is very likely that Google will upgrade this during this semester. The newer version may have some incompatibilities, so it is important that we start with this version and end with the same.



# Python Introduction


* [Anaconda v3.6](https://www.continuum.io/downloads) Scientific Python Distribution, including:
    * [Scikit-Learn](http://scikit-learn.org/)
    * [Pandas](http://pandas.pydata.org/)
    * Others: csv, json, numpy, scipy
* [Jupyter Notebooks](http://jupyter.readthedocs.io/en/latest/install.html)
* [PyCharm IDE](https://www.jetbrains.com/pycharm/)
* [Cx_Oracle](http://cx-oracle.sourceforge.net/)
* [MatPlotLib](http://matplotlib.org/)

## Jupyter Notebooks

Space matters in Python, indent code to define blocks

Jupyter Notebooks Allow Python and Markdown to coexist.

Even $\LaTeX$:

$ f'(x) = \lim_{h\to0} \frac{f(x+h) - f(x)}{h}. $

## Python Versions

* If you see `xrange` instead of `range`, you are dealing with Python 2
* If you see `print x` instead of `print(x)`, you are dealing with Python 2 

In [2]:
# What version of Python do you have?

import keras
import tensorflow as tf
import sys
import sklearn as sk
import pandas as pd
print("Tensor Flow Version: {}".format(tf.__version__))
print("Keras Version: {}".format(keras.__version__))
print()
print("Python {}".format(sys.version))
print('Pandas {}'.format(pd.__version__))
print('Scikit-Learn {}'.format(sk.__version__))

Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Tensor Flow Version: 1.4.1
Keras Version: 2.1.2

Python 3.6.3 |Anaconda custom (64-bit)| (default, Oct  6 2017, 12:04:38) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Pandas 0.20.3
Scikit-Learn 0.19.1


Software used in this class:
    
* **Python** - The programming language.
* **TensorFlow** - Googles deep learning framework, must have the version specified above. 
* **Keras** - [Keras](https://github.com/fchollet/keras) is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. 
* **Pandas** - Allows for data preprocessing.  Tutorial [here](http://pandas.pydata.org/pandas-docs/version/0.18.1/tutorials.html)
* **Scikit-Learn** - Machine learning framework for Python.  Tutorial [here](http://scikit-learn.org/stable/tutorial/basic/tutorial.html).

## Count to 10 in Python

Use a `for` loop and a `range`.

In [3]:
#Python cares about space!  No curly braces.
for x in range(1,10):  # If you ever see xrange, you are in Python 2
    print(x)  # If you ever see print x (no parenthesis), you are in Python 2


1
2
3
4
5
6
7
8
9


Printing Numbers and Strings
============================

In [4]:
sum = 0
for x in range(1,10):
    sum += x
    print("Adding {}, sum so far is {}".format(x,sum))
    
print("Final sum: {}".format(sum))

Adding 1, sum so far is 1
Adding 2, sum so far is 3
Adding 3, sum so far is 6
Adding 4, sum so far is 10
Adding 5, sum so far is 15
Adding 6, sum so far is 21
Adding 7, sum so far is 28
Adding 8, sum so far is 36
Adding 9, sum so far is 45
Final sum: 45


Lists & Sets
============

In [5]:
c = ['a', 'b', 'c', 'd']
print(c)

['a', 'b', 'c', 'd']


In [6]:
# Iterate over a collection.
for s in c:
    print(s)

a
b
c
d


In [7]:
# Iterate over a collection, and know where your index.  (Python is zero-based!)
for i,c in enumerate(c):
    print("{}:{}".format(i,c))

0:a
1:b
2:c
3:d


In [8]:
# Manually add items, lists allow duplicates
c = []
c.append('a')
c.append('b')
c.append('c')
c.append('c')
print(c)

['a', 'b', 'c', 'c']


In [9]:
# Manually add items, sets do not allow duplicates
# Sets add, lists append.  I find this annoying.
c = set()
c.add('a')
c.add('b')
c.add('c')
c.add('c')
print(c)

{'b', 'a', 'c'}


In [10]:
# Insert
c = ['a','b','c']
c.insert(0,'a0')
print(c)
# Remove
c.remove('b')
print(c)
# Remove at index
del c[0]
print(c)

['a0', 'a', 'b', 'c']
['a0', 'a', 'c']
['a', 'c']


Maps/Dictionaries/Hash Tables
=============================

In [11]:
map = { 'name': "Jeff", 'address':"123 Main"}
print(map)
print(map['name'])

if 'name' in map:
    print("Name is defined")
    
if 'age' in map:
    print("age defined")
else:
    print("age undefined")

{'name': 'Jeff', 'address': '123 Main'}
Jeff
Name is defined
age undefined


In [12]:
map = { 'name': "Jeff", 'address':"123 Main"}
# All of the keys
print("Key: {}".format(map.keys()))

# All of the values
print("Values: {}".format(map.values()))

Key: dict_keys(['name', 'address'])
Values: dict_values(['Jeff', '123 Main'])


In [13]:
# Python list & map structures 
customers = [
    {'name': 'Jeff & Tracy Heaton', 'pets': ['Wynton','Cricket']},
    {'name': 'John Smith', 'pets': ['rover']},
    {'name': 'Jane Doe'}
]

print(customers)

for customer in customers:
    print("{}:{}".format(customer['name'],customer.get('pets','no pets')))

[{'name': 'Jeff & Tracy Heaton', 'pets': ['Wynton', 'Cricket']}, {'name': 'John Smith', 'pets': ['rover']}, {'name': 'Jane Doe'}]
Jeff & Tracy Heaton:['Wynton', 'Cricket']
John Smith:['rover']
Jane Doe:no pets


# Module 1 Assignment

You can find the first assignmeht here: [assignment 1](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class1.ipynb)